Các thư viện cần thiết


In [1]:
import pandas as pd
from datetime import date, datetime
from bs4 import BeautifulSoup
import requests

Crawl dữ liệu web trên cafef.vn

Hàm crawl dữ liệu

In [2]:
def crawl_data(url):
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    # Kiểm tra yêu cầu có thành công không
    if response.status_code == 200:
        # Tạo đối tượng BeautifulSoup để phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Tìm thẻ table có id là 'tableContent'
        table = soup.find('table', id='tableContent')
        header_table = soup.find('table', id='tblGridData')
        data = []
        data_header =[]
        if header_table:
            print("Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).")
        
        # Lấy tên cột từ hàng đầu tiên của bảng tiêu đề
            header_row = header_table.find_all('td', class_='h_t')
            for header in header_row:
                data_header.append(header.get_text(strip=True))
        # Kiểm tra xem table có tồn tại không
        else:
            print("Không tìm thấy thẻ table với id 'tblGridData'.")
        if table:
            print("Đã tìm thấy thẻ table với id 'tableContent'.")
            
            # Tìm tất cả các hàng (thẻ <tr>) có class là 'r_item' hoặc 'r_item_a'
            rows = table.find_all('tr', class_=['r_item', 'r_item_a'])
            
            # Lặp qua từng hàng và lấy dữ liệu
            for row in rows:
                # Lấy tất cả các cột (thẻ <td>) trong hàng
                cols = row.find_all('td')
                
                # Tạo danh sách các giá trị văn bản từ các cột, cách nhau bởi dấu tab
                row_data = '\t'.join([col.get_text(strip=True) for col in cols])
                data.append(row_data)

        else:
            print("Không tìm thấy thẻ table với id 'tableContent'.")
    else:
        print(f"Yêu cầu không thành công: {response.status_code}")
    return data,data_header

Hàm xử lý các trường trùng nhau (dùng để merge file)

In [3]:
# Hàm để đổi tên các giá trị trùng nhau trong cột
def rename_duplicate_assets(column):
    # Tạo danh sách các giá trị đã gặp
    seen = {}
    
    # Hàm để đổi tên dựa trên việc kiểm tra giá trị đã gặp
    def rename_value(value):
        if value not in seen:
            seen[value] = 1
            return value
        else:
            seen[value] += 1
            return f"{value} ({seen[value]})"  # Thêm chỉ số vào giá trị trùng

    # Áp dụng hàm rename_value cho cột đã được truyền vào
    return column.apply(rename_value)

#Sau khi xử lý
# II.Tài sản cố định
# 1. Tài sản cố định hữu hình
# - Nguyên giá
# - Giá trị hao mòn lũy kế
# 2. Tài sản cố định thuê tài chính
# - Nguyên giá (2)
# - Giá trị hao mòn lũy kế (2)
# 3. Tài sản cố định vô hình
# - Nguyên giá (3)
# - Giá trị hao mòn lũy kế (3)
# III. Bất động sản đầu tư
# - Nguyên giá (4)
# - Giá trị hao mòn lũy kế (4)


Hàm xử lý data (loại bỏ khoảng trắng, đặt trên cột, sửa tên cột)

In [4]:
def dataframe(data,data_header):
    processed_data = []

    for row in data:
        # Tách mỗi hàng dữ liệu bằng dấu tab
        columns = row.split('\t')
        processed_data.append(columns)

    # Tạo DataFrame từ danh sách dữ liệu đã xử lý
    df = pd.DataFrame(processed_data)

    # Đặt tên cho các cột
    column_names = [f'Column {i+1}' for i in range(df.shape[1])]  # Tạo tên cột tự động nếu không có sẵn
    df.columns = column_names
    # Xóa các cột cụ thể
    df = df.drop(columns=['Column 6', 'Column 7', 'Column 8', 'Column 9', 'Column 10', 'Column 11', 'Column 12', 'Column 13'])
    df.columns.values[1:] = data_header
    
    # Lấy cột 'Column 1' từ DataFrame
    column_1 = df['Column 1']

    # Gọi hàm để đổi tên các giá trị trùng nhau
    renamed_column = rename_duplicate_assets(column_1)
    # Cập nhật lại cột trong DataFrame
    df['Column 1'] = renamed_column

    return df

Crawl báo cáo tài chính

In [5]:
# Danh sách các ký hiệu chứng khoán
symbols = ['tos','bsr','plx','pvs','pvd','pos']
years = ['2024','2023','2022','2021','2020']


In [6]:
for symbol in symbols:
    for year in years:
        # Tạo các URL cho từng năm
        url = f'https://s.cafef.vn/bao-cao-tai-chinh/{symbol}/bsheet/{year}/3/0/0/0/bao-cao-tai-chinh-.chn'
        data,data_header = crawl_data(url)
        df = dataframe(data,data_header)

        # Kiểm tra nếu DataFrame có dữ liệu và lưu vào file Excel
        if not df.empty:
            df.to_excel(f'raw_data/Baocaotaichinh/balance_{symbol}_{year}.xlsx', index=False)
            print(f"Dữ liệu cho {symbol} năm {year} đã được lưu vào file balance_{symbol}_{year}.xlsx")
        else:
            print(f"Không có dữ liệu cho {symbol} năm {year}.")


Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2024 đã được lưu vào file balance_tos_2024.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2023 đã được lưu vào file balance_tos_2023.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2022 đã được lưu vào file balance_tos_2022.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2021 đã được lưu vào file balance_tos_2021.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2020 đã được lưu vào file balance_tos_2020.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho bsr năm 2024 đã được lưu vào file balance_bsr_2024.xls

Crawl kết quả kinh doanh

In [7]:

for symbol in symbols:
    for year in years:
        # Tạo các URL cho từng năm
        url = f'https://s.cafef.vn/bao-cao-tai-chinh/{symbol}/incsta/{year}/3/0/0/0/ket-qua-hoat-dong-kinh-doanh-.chn'
        data,data_header = crawl_data(url)
        df = dataframe(data,data_header)

        # Kiểm tra nếu DataFrame có dữ liệu và lưu vào file Excel
        if not df.empty:
            df.to_excel(f'raw_data/ketquakinhdoanh/income_{symbol}_{year}.xlsx', index=False)
            print(f"Dữ liệu cho {symbol} năm {year} đã được lưu vào file balance_{symbol}_{year}.xlsx")
        else:
            print(f"Không có dữ liệu cho {symbol} năm {year}.")


Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2024 đã được lưu vào file balance_tos_2024.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2023 đã được lưu vào file balance_tos_2023.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2022 đã được lưu vào file balance_tos_2022.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2021 đã được lưu vào file balance_tos_2021.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2020 đã được lưu vào file balance_tos_2020.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho bsr năm 2024 đã được lưu vào file balance_bsr_2024.xls

Crawl lưu chuyển tiền tệ

In [8]:

for symbol in symbols:
    for year in years:
        # Tạo các URL cho từng năm
        url = f'https://s.cafef.vn/bao-cao-tai-chinh/{symbol}/cashflow/{year}/3/0/0/0/bao-cao-tai-chinh-.chn'
        data,data_header = crawl_data(url)
        df = dataframe(data,data_header)

        # Kiểm tra nếu DataFrame có dữ liệu và lưu vào file Excel
        if not df.empty:
            df.to_excel(f'raw_data/luuchuyentiente/cashflow_{symbol}_{year}.xlsx', index=False)
            print(f"Dữ liệu cho {symbol} năm {year} đã được lưu vào file cashflow_{symbol}_{year}.xlsx")
        else:
            print(f"Không có dữ liệu cho {symbol} năm {year}.")


Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2024 đã được lưu vào file cashflow_tos_2024.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2023 đã được lưu vào file cashflow_tos_2023.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2022 đã được lưu vào file cashflow_tos_2022.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2021 đã được lưu vào file cashflow_tos_2021.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho tos năm 2020 đã được lưu vào file cashflow_tos_2020.xlsx
Đã tìm thấy thẻ table với id 'tblGridData' (tiêu đề).
Đã tìm thấy thẻ table với id 'tableContent'.
Dữ liệu cho bsr năm 2024 đã được lưu vào file cashflow_bsr_20

Sử dụng thư viện VNSTOCK để lấy dữ liệu giá chứng khoán

In [9]:
from vnstock import *
# Hàm lấy giá cố phiếu
def get_stock_price(symbol):
    stock_hist = stock_historical_data(symbol, "2020-01-01",
        str(date.today()), "1D", "stock")
    stock_hist["%_change"] = round(((stock_hist['close'] - stock_hist['close'].shift(1))
                                    / stock_hist['close'].shift(1)) * 100, 2)
    stock_hist['change'] = (stock_hist['close'] - stock_hist['close'].shift(1))
    stock_hist['time'] = stock_hist['time'].astype(str)
    stock_hist = stock_hist.rename(columns={'ticker': 'stock_name', 'time': 'date'})

    return stock_hist

**Vui lòng chuyển đổi sang Vnstock3** thế hệ mới (3.1.0) với câu lệnh: `pip install vnstock3 --upgrade`.
**Từ 1/1/2025, vnstock3 sẽ được cài đặt khi sử dụng cú pháp** `pip install vnstock` **thay cho Vnstock Legacy** hiện tại.
Xem chi tiết [chuyển đổi sang vnstock3](https://vnstocks.com/docs/tai-lieu/migration-chuyen-doi-sang-vnstock3).
Phiên bản **Vnstock Legacy (0.2.9.2.2)** bạn đang sử dụng **sẽ không được nâng cấp thêm.**


In [10]:
from vnstock3 import Vnstock

stock = Vnstock().stock(symbol='TOS', source='VCI')
info_company = stock.listing.symbols_by_industries()
info_company_list =[]
company_info_list = []
for symbol in symbols:
    
    info=info_company[info_company['symbol']==symbol.upper()]
    company = Vnstock().stock(symbol=symbol.upper(), source='TCBS').company
    company_info = company.overview()
    company_info_list.append(company_info)
    info_company_list.append(info)

info_company_df = pd.concat(company_info_list,ignore_index=True)
info_company_df['symbol'] = [symbol.upper() for symbol in symbols]
company_info_df = pd.concat(info_company_list,ignore_index=True)
company_info_df = company_info_df.drop(columns=['en_organ_name','en_icb_name3','en_icb_name2','en_icb_name4','com_type_code', 'icb_code1', 'icb_code2', 'icb_code3', 'icb_code4'])

info_stock_df = pd.merge(company_info_df,info_company_df,on='symbol',how='inner')
info_stock_df

Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
2024-12-11 10:42:00 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,symbol,organ_name,icb_name3,icb_name2,icb_name4,exchange,industry,company_type,no_shareholders,foreign_percent,...,established_year,no_employees,stock_rating,delta_in_week,delta_in_month,delta_in_year,short_name,website,industry_id,industry_id_v2
0,TOS,Công ty Cổ phần Dịch vụ biển Tân Cảng,"Thiết bị, Dịch vụ và Phân phối Dầu khí",Dầu khí,Thiết bị và Dịch vụ Dầu khí,UPCOM,Dầu khí,CT,376,0.000,...,2012,882,3.0,0.023,0.047,1.025,Dịch vụ biển Tân Cảng,https://tancangoffshore.com,316,573
1,BSR,Công ty Cổ phần Lọc Hóa dầu Bình Sơn,Sản xuất Dầu khí,Dầu khí,Sản xuất và Khai thác dầu khí,UPCOM,Dầu khí,CT,32640,0.007,...,2008,1855,3.1,0.081,-0.014,0.025,Lọc Hóa dầu Bình Sơn,https://bsr.com.vn,144,533
2,PLX,Tập đoàn Xăng dầu Việt Nam,Sản xuất Dầu khí,Dầu khí,Sản xuất và Khai thác dầu khí,HOSE,Dầu khí,CT,15626,0.176,...,1995,26262,2.2,-0.010,-0.009,0.043,Petrolimex,https://www.petrolimex.com.vn,144,533
3,PVS,Tổng Công ty Cổ phần Dịch vụ Kỹ thuật Dầu khí ...,"Thiết bị, Dịch vụ và Phân phối Dầu khí",Dầu khí,Thiết bị và Dịch vụ Dầu khí,HNX,Dầu khí,CT,18979,0.203,...,2006,7209,2.3,-0.003,-0.103,-0.231,DVKT Dầu khí PTSC,http://www.ptsc.com.vn,316,573
4,PVD,Tổng Công ty Cổ phần Khoan và Dịch vụ khoan Dầ...,"Thiết bị, Dịch vụ và Phân phối Dầu khí",Dầu khí,Thiết bị và Dịch vụ Dầu khí,HOSE,Dầu khí,CT,0,0.107,...,2006,1924,2.7,0.010,-0.078,-0.293,Khoan Dầu khí PVDrilling,http://www.pvdrilling.com.vn,316,573
5,POS,"Công ty Cổ phần Dịch vụ lắp đặt, vận hành và b...","Thiết bị, Dịch vụ và Phân phối Dầu khí",Dầu khí,Thiết bị và Dịch vụ Dầu khí,UPCOM,Dầu khí,CT,339,0.003,...,2007,622,2.8,-0.023,0.004,0.528,Bảo dưỡng công trình DK biển,https://pos.ptsc.com.vn,316,573


In [11]:

stock_history = []
stock_info_company =[]
symbols = ['TOS','BSR','PLX','PVS','PVD','POS']
for symbol in symbols:
    stock_price = get_stock_price(symbol)
    stock_history.append(stock_price)
stock_history_df = pd.concat(stock_history,ignore_index=True)    


In [12]:
stock_history_df.to_excel('clean_data/history_price.xlsx')
# info_stock_df.to_excel('clean_data/info_company.xlsx')

Xử lý data (merge, remove character,tranform, ....)

Các hàm làm sạch data báo cáo tài chính và kết quả kinh doanh



In [13]:
import re

# Hàm chuyển đổi giá trị sang số nguyên
def convert_to_numeric(value):
    try:
        return int(value.replace(',', ''))
    except:
        return value  # Nếu không phải số, giữ nguyên

# Hàm loại bỏ số thứ tự
def remove_numbering(text):
    return re.sub(r'^\d+\.\s*|\b[I,V]+\.\s*', '', text)

# Hàm đổi tên cột
def rename_column(df, rename_dict):
    df['Column 1'] = df['Column 1'].replace(rename_dict)
    return df
#Hàm làm sạch dữ liệu kết quả kinh doanh   

def clean_text(text):
    # Loại bỏ nội dung trong dấu ngoặc nhọn và dấu ngoặc đơn
    text = re.sub(r'\{.*?\}|\(.*?\)', '', text)
    # Loại bỏ cụm từ "Trong đó:" và các ký hiệu không cần thiết
    text = re.sub(r'Trong đó:\s*', '', text)
    # Loại bỏ các ký hiệu không cần thiết
    text = re.sub(r'[=+\-]', '', text)
    return text.strip()
# Hàm làm sạch dữ liệu
def clean_data_balance(df):
    # Tạo từ điển đổi tên
    rename_dict = {
        "A- TÀI SẢN NGẮN HẠN": "TÀI SẢN NGẮN HẠN",
        "3. Đầu tư nắm giữ đến ngày đáo hạn":"Đầu tư ngắn hạn nắm giữ đến ngày đáo hạn",
        "IV. Hàng tồn kho":"Hàng tồn kho, ròng",
        "4. Giao dịch mua bán lại trái phiếu Chính phủ":"Giao dịch TSNH mua bán lại trái phiếu Chính phủ",
        "V.Tài sản ngắn hạn khác":"Tài Sản Ngắn Hạn Khác",
        "B. TÀI SẢN DÀI HẠN": "TÀI SẢN DÀI HẠN",
        "- Nguyên giá": "Nguyên giá TSCĐ hữu hình",
        "- Nguyên giá (2)": "Nguyên giá tài sản thuê tài chính",
        "- Nguyên giá (3)": "Nguyên giá TSCĐ vô hình",
        "- Nguyên giá (4)": "Nguyên giá tài sản đầu tư",
        "- Giá trị hao mòn lũy kế": "Khấu hao lũy kế TSCĐ hữu hình",
        "- Giá trị hao mòn lũy kế (2)": "Khấu hao lũy kế tài sản thuê tài chính",
        "- Giá trị hao mòn lũy kế (3)": "Khấu hao lũy kế TSCĐ vô hình",
        "- Giá trị hao mòn lũy kế (4)": "Khấu hao lũy kế tài sản đầu tư",
        "TỔNG CỘNG TÀI SẢN": "TỔNG CỘNG TÀI SẢN",
        "4. Tài sản dài hạn khác": "Các tài sản dài hạn khác",
        "C. NỢ PHẢI TRẢ": "NỢ PHẢI TRẢ",
        "10. Cổ phiếu ưu đãi":"Cổ phiếu ưu đãi nợ dài hạn",
        "D.VỐN CHỦ SỞ HỮU": "VỐN CHỦ SỞ HỮU",
        "- Cổ phiếu phổ thông có quyền biểu quyết":"Cổ phiếu phổ thông có quyền biểu quyết",
        "- Cổ phiếu ưu đãi":"Cổ phiếu ưu đãi",
        "- LNST chưa phân phối lũy kế đến cuối kỳ trước": "LNST chưa phân phối lũy kế đến cuối kỳ trước",
        "- LNST chưa phân phối kỳ này": "LNST chưa phân phối lũy kỳ này",
        "TỔNG CỘNG NGUỒN VỐN": "TỔNG CỘNG NGUỒN VỐN"
    }
    df = rename_column(df, rename_dict)
    # Áp dụng hàm loại bỏ số thứ tự
    df["Column 1"] = df["Column 1"].apply(remove_numbering)
    # Loại bỏ hàng có giá trị "TÀI SẢN"
    df = df[df['Column 1'] != "TÀI SẢN"]
    df = df[df['Column 1'] != "NGUỒN VỐN"]
    # Chuyển đổi các giá trị thành số nguyên
    df = df.applymap(convert_to_numeric)
    
    return df
def clean_data_income(df):
    df["Column 1"] = df["Column 1"].apply(remove_numbering)
    # Chuyển đổi các giá trị thành số nguyên
    df = df.applymap(convert_to_numeric)
    df["Column 1"] = df["Column 1"].apply(clean_text)
    return df
def delete_row(df):
        # Lọc bỏ các dòng mà 'Column 1' chứa các cụm từ không mong muốn
    unwanted_strings = ['I. Lưu chuyển tiền từ hoạt động kinh doanh', 
                    'II. Lưu chuyển tiền từ hoạt động đầu tư', 
                    'III. Lưu chuyển tiền từ hoạt động tài chính']

    df = df[~df['Column 1'].isin(unwanted_strings)]
    return df
def clean_data_cashflow(df):
    df= delete_row(df)
    df["Column 1"] = df["Column 1"].apply(remove_numbering)
    # Chuyển đổi các giá trị thành số nguyên
    df = df.applymap(convert_to_numeric)
    df["Column 1"] = df["Column 1"].apply(clean_text)
    return df




Merge dữ liệu

In [14]:
def merge_excel_files(file_list):
    """
    Hàm để merge các file Excel dựa trên cột "TÀI SẢN".
    
    Tham số:
    file_list (list): Danh sách đường dẫn tới các file Excel.
    
    Trả về:
    DataFrame: DataFrame chứa dữ liệu đã merge từ các file.
    """
    # Khởi tạo một biến DataFrame rỗng
    df_merged = pd.DataFrame()

    # Duyệt qua từng file trong danh sách
    for i, file in enumerate(file_list):
        # Đọc file Excel
        df = pd.read_excel(file)

        # Nếu là file đầu tiên, gán nó cho df_merged
        if df_merged.empty:
            df_merged = df
        else:
            # Merge các DataFrame lại với nhau dựa trên cột "TÀI SẢN"
            df_merged = df_merged.merge(df, on='Column 1', how='outer')

    return df_merged
# Danh sách các ký hiệu
symbols = ['tos','bsr','plx','pvc','oil','pvs','pvd','pvt','pos']

# Duyệt qua từng ký hiệu và gọi hàm
for symbol in symbols:
    file_list_balance = [
        f'raw_data/Baocaotaichinh/balance_{symbol}_2021.xlsx',
        f'raw_data/Baocaotaichinh/balance_{symbol}_2022.xlsx',
        f'raw_data/Baocaotaichinh/balance_{symbol}_2023.xlsx',
        f'raw_data/Baocaotaichinh/balance_{symbol}_2024.xlsx'
    ]
    file_list_income = [
        f'raw_data/ketquakinhdoanh/income_{symbol}_2021.xlsx',
        f'raw_data/ketquakinhdoanh/income_{symbol}_2022.xlsx',
        f'raw_data/ketquakinhdoanh/income_{symbol}_2023.xlsx',
        f'raw_data/ketquakinhdoanh/income_{symbol}_2024.xlsx'
    ]
    file_list_cashflow =[
        f'raw_data/luuchuyentiente/cashflow_{symbol}_2021.xlsx',
        f'raw_data/luuchuyentiente/cashflow_{symbol}_2022.xlsx',
        f'raw_data/luuchuyentiente/cashflow_{symbol}_2023.xlsx',
        f'raw_data/luuchuyentiente/cashflow_{symbol}_2024.xlsx'
    ]
    # Gọi hàm và lưu kết quả vào biến df
    df_merged_balance = merge_excel_files(file_list_balance)
    df_merged_income = merge_excel_files(file_list_income)
    df_merged_cashflow = merge_excel_files(file_list_cashflow)
    
    # Gọi hàm làm sạch dữ liệu
    df_cleaned_balance = clean_data_balance(df_merged_balance)
    df_cleaned_income = clean_data_income(df_merged_income)
    df_cleaned_cashflow = clean_data_cashflow(df_merged_cashflow)
    # Lưu kết quả thành file Excel với tên theo ký hiệu
    df_cleaned_balance.to_excel(f'clean_data/balance/balance_{symbol}.xlsx', index=False)
    df_cleaned_income.to_excel(f'clean_data/income/income_{symbol}.xlsx', index=False)
    df_cleaned_cashflow.to_excel(f'clean_data/cashflow/cashflow_{symbol}.xlsx', index=False)
    # In thông báo hoàn thành
    print(f'Đã lưu file: balance_{symbol}.xlsx')
    print(f'Đã lưu file: income_{symbol}.xlsx')
    print(f'Đã lưu file: cashflow_{symbol}.xlsx')
    


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Đã lưu file: balance_tos.xlsx
Đã lưu file: income_tos.xlsx
Đã lưu file: cashflow_tos.xlsx



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Đã lưu file: balance_bsr.xlsx
Đã lưu file: income_bsr.xlsx
Đã lưu file: cashflow_bsr.xlsx



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Đã lưu file: balance_pvd.xlsx
Đã lưu file: income_pvd.xlsx
Đã lưu file: cashflow_pvd.xlsx



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Đã lưu file: balance_pvt.xlsx
Đã lưu file: income_pvt.xlsx
Đã lưu file: cashflow_pvt.xlsx
Đã lưu file: balance_pos.xlsx
Đã lưu file: income_pos.xlsx
Đã lưu file: cashflow_pos.xlsx



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Chuẩn bị dữ liệu

In [15]:
import pandas as pd
stock_relation = pd.read_excel('./stock_relation.xlsx', sheet_name='dim_account')
account_relation = pd.read_excel('./stock_relation.xlsx', sheet_name='account_relation')
stock_cashflow_relation = pd.read_excel('./stock_relation.xlsx', sheet_name='dim_account_cashflow')
account_cashflow_relation = pd.read_excel('./stock_relation.xlsx', sheet_name='account_cashflow_relation')

In [16]:
Account_Balance =stock_relation[stock_relation['account_type']=='cdkt']
Account_Balance

,account_key,account_name,sign,account_type,level
0,100,TÀI SẢN NGẮN HẠN,1,cdkt,2
1,110,Tiền và các khoản tương đương tiền,1,cdkt,3
2,111,Tiền,1,cdkt,4
3,112,Các khoản tương đương tiền,1,cdkt,4
4,120,Các khoản đầu tư tài chính ngắn hạn,1,cdkt,3
...,...,...,...,...,...
112,429,Lợi ích cổ đông không kiểm soát,1,cdkt,4
113,430,Nguồn kinh phí và quỹ khác,1,cdkt,3
114,431,Nguồn kinh phí,1,cdkt,4
115,432,Nguồn kinh phí đã hình thành TSCĐ,1,cdkt,4


In [17]:
Account_income = stock_relation[stock_relation['account_type']=='kqkd']
Account_income

,account_key,account_name,sign,account_type,level
117,01,Doanh thu bán hàng và cung cấp dịch vụ,1,kqkd,1
118,02,Các khoản giảm trừ doanh thu,-1,kqkd,1
119,10,Doanh thu thuần về bán hàng và cung cấp dịch vụ,1,kqkd,1
120,11,Giá vốn hàng bán,-1,kqkd,1
121,20,Lợi nhuận gộp về bán hàng và cung cấp dịch vụ,1,kqkd,1
122,21,Doanh thu hoạt động tài chính,1,kqkd,1
123,22,Chi phí tài chính,-1,kqkd,1
124,23,Chi phí lãi vay,-1,kqkd,2
125,24,"Phần lãi lỗ trong công ty liên doanh, liên kết",1,kqkd,1
126,25,Chi phí bán hàng,-1,kqkd,1


In [18]:
Account_cashflow =stock_cashflow_relation
Account_cashflow

,account_key,account_name,sign,account_type,level
0,1100,Lợi nhuận trước thuế,1,lctt,2
1,1200,Điều chỉnh cho các khoản,1,lctt,2
2,1210,Khấu hao TSCĐ và BĐSĐT,1,lctt,3
3,1220,Các khoản dự phòng,1,lctt,3
4,1230,"Lãi, lỗ chênh lệch tỷ giá hối đoái do đánh giá...",1,lctt,3
5,1240,"Lãi, lỗ từ hoạt động đầu tư",-1,lctt,3
6,1250,Chi phí lãi vay,1,lctt,3
7,1260,Các khoản điều chỉnh khác,1,lctt,3
8,1300,Lợi nhuận từ hoạt động kinh doanh trước thay đ...,1,lctt,2
9,1310,"Tăng, giảm các khoản phải thu",1,lctt,3


In [19]:
# Hàm chuyển đổi từ Quý sang ngày đầu tháng

def convert_quarter_to_date(quarter_str):
    quarter, year = quarter_str.split('-')
    year = int(float(year))
    if quarter == 'Quý 1':
        return f"{year}-01-01"
    elif quarter == 'Quý 2':
        return f"{year}-04-01"
    elif quarter == 'Quý 3':
        return f"{year}-07-01"
    elif quarter == 'Quý 4':
        return f"{year}-10-01"
    else:
        raise ValueError(f"Invalid quarter: {quarter}")

Hàm tranform các file dữ liệu

In [20]:
def transform_data(stock, report_type):
    df = pd.read_excel(f'clean_data/{report_type}/{report_type}_{stock}.xlsx')
    if(report_type == 'balance'):
        df = pd.merge(df, stock_relation, left_on='Column 1', right_on='account_name', how='left')
        df = df.drop(columns=['Column 1', 'account_name', 'sign', 'account_type', 'level'])
        df = df.melt(id_vars=['account_key'], var_name='quarter', value_name='End_of_Period')
        df['date'] = df['quarter'].apply(convert_quarter_to_date)
        df = df.drop(columns=['quarter'])
        df['stock_name'] = f'{stock}'.upper()
    elif(report_type == 'income'):
        df = pd.merge(df, stock_relation, left_on='Column 1', right_on='account_name', how='left')
        df = df.drop(columns=['Column 1', 'account_name', 'sign', 'account_type', 'level'])
        df = df.melt(id_vars=['account_key'], var_name='quarter', value_name='End_of_Period')
        df['date'] = df['quarter'].apply(convert_quarter_to_date)
        df = df.drop(columns=['quarter'])
        df['stock_name'] = f'{stock}'.upper()
    elif(report_type=='cashflow'):
        df = pd.merge(df, stock_cashflow_relation, left_on='Column 1', right_on='account_name', how='left')
        df = df.drop(columns=['Column 1', 'account_name', 'sign', 'account_type', 'level'])
        df = df.melt(id_vars=['account_key'], var_name='quarter', value_name='End_of_Period')
        df['date'] = df['quarter'].apply(convert_quarter_to_date)
        df = df.drop(columns=['quarter'])
        df['stock_name'] = f'{stock}'.upper()
    else:
        print('Lỗi không có đường dẫn đúng!')
    
    return df

In [21]:
stocks= ['tos','bsr','plx','pvs','pvd','pos']
bs_list = []
income_list = []
cashflow_list = []
for stock in stocks:
    balance_sheet = transform_data(stock,'balance')
    income_sheet = transform_data(stock,'income')
    cashflow_sheet = transform_data(stock,'cashflow')
    bs_list.append(balance_sheet)
    income_list.append(income_sheet)
    cashflow_list.append(cashflow_sheet)
    
bs_df = pd.concat(bs_list,ignore_index=True)
income_df = pd.concat(income_list, ignore_index=True)
cashflow_df= pd.concat(cashflow_list,ignore_index=True)

Các dữ liệu chuẩn bị

In [22]:

bs_df['End_of_Period'] = bs_df['End_of_Period'].fillna(0)
bs_df = bs_df.drop_duplicates()
# Tìm trùng lặp dựa trên cột 'Name' và 'PhoneNumber'
duplicates = bs_df[bs_df.duplicated(subset=['account_key', 'date','stock_name'])]
bs_df.head()

,account_key,End_of_Period,date,stock_name
0,100,5.763448e+11,2020-10-01,TOS
1,110,8.578630e+10,2020-10-01,TOS
2,111,7.028630e+10,2020-10-01,TOS
3,112,1.550000e+10,2020-10-01,TOS
4,120,0.000000e+00,2020-10-01,TOS


In [23]:

income_df['End_of_Period'] = income_df['End_of_Period'].fillna(0)
income_df.head()

,account_key,End_of_Period,date,stock_name
0,01,2.974283e+11,2020-10-01,TOS
1,02,2.396652e+08,2020-10-01,TOS
2,10,2.971886e+11,2020-10-01,TOS
3,11,2.267051e+11,2020-10-01,TOS
4,20,7.048348e+10,2020-10-01,TOS


In [24]:
cashflow_df.to_excel('cashflow.xlsx')
cashflow_df['End_of_Period'] = cashflow_df['End_of_Period'].fillna(0)

In [25]:
Account_Balance.head()


,account_key,account_name,sign,account_type,level
0,100,TÀI SẢN NGẮN HẠN,1,cdkt,2
1,110,Tiền và các khoản tương đương tiền,1,cdkt,3
2,111,Tiền,1,cdkt,4
3,112,Các khoản tương đương tiền,1,cdkt,4
4,120,Các khoản đầu tư tài chính ngắn hạn,1,cdkt,3


In [26]:
Account_income.head(20)

,account_key,account_name,sign,account_type,level
117,01,Doanh thu bán hàng và cung cấp dịch vụ,1,kqkd,1
118,02,Các khoản giảm trừ doanh thu,-1,kqkd,1
119,10,Doanh thu thuần về bán hàng và cung cấp dịch vụ,1,kqkd,1
120,11,Giá vốn hàng bán,-1,kqkd,1
121,20,Lợi nhuận gộp về bán hàng và cung cấp dịch vụ,1,kqkd,1
122,21,Doanh thu hoạt động tài chính,1,kqkd,1
123,22,Chi phí tài chính,-1,kqkd,1
124,23,Chi phí lãi vay,-1,kqkd,2
125,24,"Phần lãi lỗ trong công ty liên doanh, liên kết",1,kqkd,1
126,25,Chi phí bán hàng,-1,kqkd,1


In [27]:

stock_history_df.head()
stock_history_df = stock_history_df.rename(columns={
    'open': 'open_price',
    'high': 'high_price',
    'low': 'low_price',
    'close': 'close_price'
})
stock_history_df['date']=pd.to_datetime(stock_history_df['date'])
stock_history_df.to_excel('test.xlsx')

In [33]:
stock_history_df[stock_history_df['stock_name']=='PLX'].tail()

,date,open_price,high_price,low_price,close_price,volume,stock_name,%_change,change
3154,2024-12-04,40050,40900,40050,40300,651300,PLX,0.75,300.0
3155,2024-12-05,40300,40700,40000,40600,716100,PLX,0.74,300.0
3156,2024-12-06,40600,40750,40100,40450,787600,PLX,-0.37,-150.0
3157,2024-12-09,40450,40450,40000,40150,573300,PLX,-0.74,-300.0
3158,2024-12-10,40150,40450,40100,40300,284700,PLX,0.37,150.0


In [28]:
info_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symbol             6 non-null      object 
 1   organ_name         6 non-null      object 
 2   icb_name3          6 non-null      object 
 3   icb_name2          6 non-null      object 
 4   icb_name4          6 non-null      object 
 5   exchange           6 non-null      object 
 6   industry           6 non-null      object 
 7   company_type       6 non-null      object 
 8   no_shareholders    6 non-null      int64  
 9   foreign_percent    6 non-null      float64
 10  outstanding_share  6 non-null      float64
 11  issue_share        6 non-null      float64
 12  established_year   6 non-null      object 
 13  no_employees       6 non-null      int64  
 14  stock_rating       6 non-null      float64
 15  delta_in_week      6 non-null      float64
 16  delta_in_month     6 non-null 

ĐỔ dữ liệu vào SQL SERVER

In [ ]:
from sqlalchemy import create_engine

# Tạo engine kết nối
engine = create_engine("mssql+pyodbc://@LAPTOP-1VLE3QB3\\SQLEXPRESS/Stock_new?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

# # Chèn dữ liệu vào bảng 'Account_income'
# Account_income.to_sql('Account_income', con=engine, if_exists='append', index=False)
# # Chèn dữ liệu vào bảng 'Account_balance'
# Account_Balance.to_sql('Account_Balance',con=engine,if_exists='append', index=False)
# #Chèn dữ liệu vào bảng 'Account_cashflow'
# Account_cashflow.to_sql('Account_cashflow',con=engine,if_exists='append', index=False)
# # Chèn dữ liệu vào bảng 'info_stock'
# info_stock_df.to_sql('info_stock', con=engine, if_exists='append', index=False)
# # Chèn dữ liệu vào bảng 'history_price'
stock_history_df.to_sql('history_price', con=engine, if_exists='append', index=False)
# # Chèn dữ liệu vào bảng 'Income'
# income_df.to_sql('Income', con=engine, if_exists='append', index=False)
# # Chèn dữ liệu vào bảng 'cashflow'
# cashflow_df.to_sql('cashflow', con=engine, if_exists='append', index=False)
# #Chèn dữ liệu vào bảng 'Balance'
# bs_df.to_sql('Balance', con=engine, if_exists='append', index=False)

-1